# Preprocessing Guide: DEM Delineation

This notebook provides a guide to using the `WatershedDelineator` tool in the CHS SDK. This powerful tool automates the process of analyzing a Digital Elevation Model (DEM) to determine drainage patterns and delineate watershed boundaries, which is a fundamental step in setting up most hydrological models.

## 1. The Hydrological Analysis Workflow

The process of delineating a watershed from a DEM involves a standard sequence of GIS operations:

1.  **Fill Sinks**: Raw DEMs often have small imperfections that create artificial depressions where water would get stuck. This step fills these sinks to ensure a continuous flow path across the landscape.
2.  **Flow Direction**: For each cell in the DEM, this step determines which of its 8 neighbors water will flow to, based on the steepest descent.
3.  **Flow Accumulation**: This step calculates, for each cell, how many upstream cells flow into it. This is used to identify the stream network (cells with high accumulation values).
4.  **Delineation**: Using the flow direction grid, this step identifies all the cells that flow into a specified **outlet point**, defining the watershed boundary.

The `WatershedDelineator` automates this entire workflow.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.preprocessing.delineation import WatershedDelineator

# Helper function for plotting grids
def plot_grid(grid, title):
    plt.figure(figsize=(7, 7))
    plt.imshow(grid, cmap='viridis')
    plt.title(title)
    plt.colorbar(label='Value')
    plt.show()

## 2. Code Example

We will use a small, synthetic DEM for this example to make the results easy to see.

In [ ]:
# 1. Create a synthetic DEM
# This represents a simple V-shaped valley sloping towards the bottom right
dem = np.array([
    [10, 10, 10, 10, 10, 10, 10, 10],
    [10,  9,  9,  9,  9,  9,  9, 10],
    [10,  9,  8,  8,  8,  8,  9, 10],
    [10,  9,  8,  7,  7,  8,  9, 10],
    [10,  9,  8,  7,  6,  7,  9, 10],
    [10,  9,  8,  8,  7,  8,  9, 10],
    [10,  9,  9,  9,  8,  9,  9, 10],
    [10, 10, 10, 10,  9, 10, 10, 10],
])

# Add a sink (a hole) to demonstrate the sink-filling process
dem[4, 2] = 5

plot_grid(dem, "Original Synthetic DEM (with sink at [4, 2])")

In [ ]:
# 2. Initialize the Delineator
delineator = WatershedDelineator(dem)

# 3. Run the preprocessing steps
# This will automatically fill sinks, and calculate flow direction and accumulation
delineator._preprocess_dem()

# Let's visualize the intermediate products
plot_grid(delineator.filled_dem, "1. Filled DEM")
plot_grid(delineator.fdr, "2. Flow Direction (D8 Method)")
plot_grid(np.log1p(delineator.fac), "3. Flow Accumulation (log scale)") # Use log scale for better viz

### Analysis of Intermediate Steps

- **Filled DEM**: Notice that the sink at `[4, 2]` has been filled (its value raised from 5 to 8) to ensure water can flow out of it.
- **Flow Direction**: The grid shows the direction of flow for each cell using the D8 standard (1=E, 2=SE, 4=S, 8=SW, etc.). You can see the flow paths converging towards the main valley line.
- **Flow Accumulation**: The bright line shows the main channel where flow accumulates. The value of each cell in this grid represents the number of upstream cells that drain into it.

## 4. Delineating the Watershed

Now that the DEM is preprocessed, we can define an outlet point and delineate its entire upstream catchment area.

In [ ]:
# Define the outlet point (bottom of the valley)
outlet_point = (4, 4)

# Delineate the watershed for this outlet
zones = delineator.delineate_parameter_zones(outlet_points=[outlet_point])
watershed_mask = zones[0].mask

# Visualize the delineated watershed
# We'll plot the original DEM but only show the cells inside the mask
watershed_dem = np.where(watershed_mask, dem, np.nan) # Set cells outside the mask to NaN

plt.figure(figsize=(8, 8))
plt.imshow(watershed_dem, cmap='terrain')
plt.plot(outlet_point[1], outlet_point[0], 'r*', markersize=15, label='Outlet Point') # Note: plot uses (x, y) which is (col, row)
plt.title('Delineated Watershed')
plt.legend()
plt.colorbar(label='Elevation')
plt.show()

The final plot shows the boundary of the watershed that drains to our specified outlet point. This boolean mask can then be used to extract parameters or run hydrological models for only this specific area. This notebook demonstrates how the `WatershedDelineator` provides a powerful, automated workflow for a critical preprocessing task in hydrology.